In [1]:
from tools import *
from data import *
from ensemble import train_ensemble, show_top_weights
import json

In [2]:
with open('params_xgb.json', 'r') as file:
    xgb_params = json.load(file)

with open('params_rf.json', 'r') as file: 
    rf_params = json.load(file)

xgb_selected_features = ['x4', 'x8', 'x9', 'x10', 'x11']
gnb_selected_features = ['x2', 'x3', 'x4', 'x6', 'x8', 'x9', 'x10', 'x11']

In [3]:
models = [
    Model(model_type='xgb', selected_features=xgb_selected_features, xgb_params=xgb_params),
    Model(model_type='gnb', selected_features=gnb_selected_features),
    Model(model_type='nn', nn_params=X_train.shape[1]),
    Model(model_type='rf', rf_params=rf_params)
]

In [4]:
results, best_result = train_ensemble(
    X_train, y_train, 
    models = models,
    k_fold_type = 'shuffle_split',
    mean_type = 'arithmetic'
)

show_top_weights(results, n_top=100)


105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step
Model trainings successful, proceeding to weight combination search...
Weight combination search done.
Top 100 Accuracies:
1. Weights: [0.1, 0.1, 0.4, 0.4] | Mean Accuracy: 0.696 | Std Accuracy: 0.007 | Mean Log Loss: 0.716 | Std Log Loss: 0.008
2. Weights: [0.1, 0.2, 0.33, 0.37] | Mean Accuracy: 0.696 | Std Accuracy: 0.006 | Mean Log Loss: 0.717 | Std Log Loss: 0.007
3. Weights: [0.1, 0.23, 0.33, 0.33] | Mean Accuracy: 0.696 | Std Accuracy: 0.006 | Mean Log Loss: 0.716 | Std Log Loss: 0.007
4. Weights: [0.07, 0.13, 0.4, 0.4] | Mean Accuracy: 0.695 | Std Accuracy: 0.008 | Mean Log Loss: 0.717 | Std Log Loss: 0.008
5. Weights: [0.07, 0.1, 0.4, 0.43] | Mean Accuracy: 0.695 | Std Accuracy: 0.008 | Mean Log Loss: 0.718 | Std Log Loss: 0.008
6. Weights: [0.2, 0.2, 0.3, 0.3] | Mea

In [5]:
results, best_result = train_ensemble(
    X_train, y_train, 
    models = models,
    k_fold_type = 'shuffle_split',
    mean_type = 'geometric'
)

show_top_weights(results, n_top=100)


105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step
Model trainings successful, proceeding to weight combination search...
Weight combination search done.
Top 100 Accuracies:
1. Weights: [0.0, 0.1, 0.43, 0.47] | Mean Accuracy: 0.700 | Std Accuracy: 0.005 | Mean Log Loss: 0.691 | Std Log Loss: 0.006
2. Weights: [0.07, 0.13, 0.4, 0.4] | Mean Accuracy: 0.700 | Std Accuracy: 0.005 | Mean Log Loss: 0.688 | Std Log Loss: 0.006
3. Weights: [0.1, 0.1, 0.37, 0.43] | Mean Accuracy: 0.700 | Std Accuracy: 0.005 | Mean Log Loss: 0.689 | Std Log Loss: 0.005
4. Weights: [0.1, 0.1, 0.4, 0.4] | Mean Accuracy: 0.700 | Std Accuracy: 0.005 | Mean Log Loss: 0.687 | Std Log Loss: 0.006
5. Weights: [0.07, 0.1, 0.4, 0.43] | Mean Accuracy: 0.700 | Std Accuracy: 0.005 | Mean Log Loss: 0.689 | Std Log Loss: 0.006
6. Weights: [0.03, 0.03, 0.47, 0.47] | 